# Day13_0 별첨: SVR (Support Vector Regression)

## 📚 학습 목표

**Part 1: 기초 - SVR 개념 이해**
1. SVR(Support Vector Regression)의 핵심 원리 이해하기
2. Epsilon-Insensitive Tube(ε-tube) 개념 익히기
3. 선형 회귀와 SVR의 차이점 이해하기
4. SVR의 핵심 파라미터(C, epsilon) 역할 파악하기

**Part 2: 심화 - SVR 활용 및 최적화**
1. RBF 커널과 gamma 파라미터 활용하기
2. 데이터 스케일링의 중요성 이해하기
3. GridSearchCV로 하이퍼파라미터 튜닝하기
4. SVR과 다른 회귀 모델 성능 비교하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| SVR | 비선형 회귀 | 주가 예측, 센서 데이터 분석 |
| Epsilon Tube | 오차 허용 범위 설정 | 노이즈가 있는 데이터 처리 |
| 커널 트릭 | 고차원 패턴 학습 | 복잡한 비선형 관계 모델링 |
| 하이퍼파라미터 튜닝 | 최적 성능 달성 | 실무 모델 배포 |

**분석가 관점**: SVR은 이상치에 강건하고 비선형 패턴을 잘 학습합니다. 특히 중소규모 데이터셋에서 뛰어난 성능을 보여, 금융/제조/센서 데이터 분석에 자주 활용됩니다!

---

## 환경 설정

In [ ]:
# 필수 라이브러리 임포트
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# sklearn 모델
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor

# sklearn 유틸리티
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.datasets import fetch_california_housing

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

# 랜덤 시드 고정
np.random.seed(42)

print("환경 설정 완료!")

## 데이터셋 준비

In [ ]:
# California Housing 데이터 로드
housing = fetch_california_housing()
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df['MedHouseVal'] = housing.target

# SVR은 계산 비용이 높으므로 샘플링
df_sample = df.sample(n=3000, random_state=42)

X = df_sample.drop('MedHouseVal', axis=1)
y = df_sample['MedHouseVal']

# 학습/테스트 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"학습 데이터: {X_train.shape[0]}개")
print(f"테스트 데이터: {X_test.shape[0]}개")

In [ ]:
# 스케일링 (SVR에 필수!)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("스케일링 완료!")
print(f"스케일링 전 X_train 범위: [{X_train.values.min():.2f}, {X_train.values.max():.2f}]")
print(f"스케일링 후 X_train 범위: [{X_train_scaled.min():.2f}, {X_train_scaled.max():.2f}]")

---

# Part 1: 기초 - SVR 개념 이해

---

## 1.1 SVR (Support Vector Regression)이란?

SVR은 SVM(Support Vector Machine)을 회귀 문제에 적용한 것입니다.

### 핵심 아이디어

- **일반 회귀**: 모든 오차를 최소화
- **SVR**: **ε(epsilon) 이내의 오차는 무시**, ε를 벗어난 오차만 최소화

이 "관용적" 접근 방식이 이상치에 강건한 모델을 만듭니다.

## 1.2 Epsilon-Insensitive Tube (ε-tube)

SVR의 핵심 개념은 **ε-tube**입니다:

```
        -------- 상한 (f(x) + ε)
        |      |
  ------+------+------ 예측선 f(x)
        |      |
        -------- 하한 (f(x) - ε)
```

- **튜브 안의 점**: 오차 = 0 (페널티 없음)
- **튜브 밖의 점**: 튜브로부터의 거리만큼 페널티

### epsilon의 효과

| epsilon | 튜브 폭 | 모델 특성 |
|---------|--------|----------|
| 작은 값 | 좁음 | 더 많은 점이 오차로 계산, 민감한 모델 |
| 큰 값 | 넓음 | 더 많은 오차 허용, 일반화된 모델 |

In [ ]:
# epsilon 시각화를 위한 간단한 데이터 생성
np.random.seed(42)
X_simple = np.linspace(0, 10, 50).reshape(-1, 1)
y_simple = 2 * X_simple.ravel() + 3 + np.random.normal(0, 1.5, 50)

# 다양한 epsilon 값으로 SVR 학습
epsilons = [0.1, 0.5, 1.0, 2.0]
fig = make_subplots(rows=2, cols=2, subplot_titles=[f'epsilon = {e}' for e in epsilons])

for idx, eps in enumerate(epsilons):
    row = idx // 2 + 1
    col = idx % 2 + 1
    
    # SVR 학습
    svr = SVR(kernel='linear', epsilon=eps, C=1.0)
    svr.fit(X_simple, y_simple)
    y_pred = svr.predict(X_simple)
    
    # 데이터 점
    fig.add_trace(
        go.Scatter(x=X_simple.ravel(), y=y_simple, mode='markers', name='데이터',
                   marker=dict(size=6, color='blue'), showlegend=(idx==0)),
        row=row, col=col
    )
    
    # 예측선
    fig.add_trace(
        go.Scatter(x=X_simple.ravel(), y=y_pred, mode='lines', name='예측',
                   line=dict(color='red', width=2), showlegend=(idx==0)),
        row=row, col=col
    )
    
    # ε-tube 상한/하한
    fig.add_trace(
        go.Scatter(x=X_simple.ravel(), y=y_pred + eps, mode='lines', name='ε-tube',
                   line=dict(color='green', dash='dash'), showlegend=(idx==0)),
        row=row, col=col
    )
    fig.add_trace(
        go.Scatter(x=X_simple.ravel(), y=y_pred - eps, mode='lines',
                   line=dict(color='green', dash='dash'), showlegend=False),
        row=row, col=col
    )

fig.update_layout(title='Epsilon에 따른 ε-tube 변화', height=600)
fig.show()

### 💡 실무 예시: epsilon 선택

- **노이즈가 많은 데이터**: epsilon을 크게 (노이즈 허용)
- **정밀 예측 필요**: epsilon을 작게
- **일반적인 시작값**: 타겟 표준편차의 10-20%

---

## 1.3 핵심 파라미터: C (Cost)

**C**는 오차에 대한 페널티 강도를 조절합니다:

| C 값 | 의미 | 결과 |
|------|------|------|
| 작은 C | 오차 허용적 | 더 평활한 모델, 과소적합 가능 |
| 큰 C | 오차에 엄격 | 더 복잡한 모델, 과적합 가능 |

In [ ]:
# C 값에 따른 모델 변화 시각화
C_values = [0.1, 1, 10, 100]
fig = make_subplots(rows=2, cols=2, subplot_titles=[f'C = {c}' for c in C_values])

for idx, C in enumerate(C_values):
    row = idx // 2 + 1
    col = idx % 2 + 1
    
    # SVR 학습
    svr = SVR(kernel='rbf', C=C, epsilon=0.5)
    svr.fit(X_simple, y_simple)
    y_pred = svr.predict(X_simple)
    
    # 데이터 점
    fig.add_trace(
        go.Scatter(x=X_simple.ravel(), y=y_simple, mode='markers',
                   marker=dict(size=6, color='blue'), showlegend=False),
        row=row, col=col
    )
    
    # 예측선
    fig.add_trace(
        go.Scatter(x=X_simple.ravel(), y=y_pred, mode='lines',
                   line=dict(color='red', width=2), showlegend=False),
        row=row, col=col
    )
    
    # 서포트 벡터 표시
    sv_idx = svr.support_
    fig.add_trace(
        go.Scatter(x=X_simple[sv_idx].ravel(), y=y_simple[sv_idx], mode='markers',
                   marker=dict(size=10, color='orange', symbol='circle-open', line=dict(width=2)),
                   name=f'Support Vectors ({len(sv_idx)})', showlegend=False),
        row=row, col=col
    )

fig.update_layout(title='C에 따른 모델 변화 (주황색: 서포트 벡터)', height=600)
fig.show()

---

## 1.4 기본 SVR 학습

In [ ]:
# 기본 SVR 모델 학습
svr_basic = SVR(kernel='rbf', C=1.0, epsilon=0.1)
svr_basic.fit(X_train_scaled, y_train)

# 예측
y_pred_svr = svr_basic.predict(X_test_scaled)

# 성능 평가
print("=== 기본 SVR 성능 ===")
print(f"R2 Score: {r2_score(y_test, y_pred_svr):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_svr)):.4f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_svr):.4f}")
print(f"\n서포트 벡터 수: {len(svr_basic.support_)}개 / {len(X_train)}개")

---

# Part 2: 심화 - SVR 활용 및 최적화

---

## 2.1 커널(Kernel)과 gamma 파라미터

SVR은 **커널 트릭**으로 비선형 패턴을 학습합니다.

### 주요 커널

| 커널 | 특성 | 사용 시점 |
|------|------|----------|
| linear | 선형 결정 경계 | 선형 관계, 고차원 데이터 |
| rbf (기본) | 가우시안 방사 기저 함수 | 비선형 관계, 일반적 선택 |
| poly | 다항식 | 다항 관계 |

### gamma 파라미터 (RBF 커널)

- **gamma**: 단일 학습 샘플의 영향 범위
- **작은 gamma**: 넓은 영향 범위, 더 평활한 경계
- **큰 gamma**: 좁은 영향 범위, 더 복잡한 경계

In [ ]:
# gamma에 따른 모델 변화 시각화
gamma_values = [0.01, 0.1, 1, 10]
fig = make_subplots(rows=2, cols=2, subplot_titles=[f'gamma = {g}' for g in gamma_values])

for idx, gamma in enumerate(gamma_values):
    row = idx // 2 + 1
    col = idx % 2 + 1
    
    # SVR 학습
    svr = SVR(kernel='rbf', C=1, gamma=gamma, epsilon=0.5)
    svr.fit(X_simple, y_simple)
    
    # 예측용 더 촘촘한 X 생성
    X_plot = np.linspace(0, 10, 200).reshape(-1, 1)
    y_pred = svr.predict(X_plot)
    
    # 데이터 점
    fig.add_trace(
        go.Scatter(x=X_simple.ravel(), y=y_simple, mode='markers',
                   marker=dict(size=6, color='blue'), showlegend=False),
        row=row, col=col
    )
    
    # 예측선
    fig.add_trace(
        go.Scatter(x=X_plot.ravel(), y=y_pred, mode='lines',
                   line=dict(color='red', width=2), showlegend=False),
        row=row, col=col
    )

fig.update_layout(title='gamma에 따른 RBF 커널 모델 변화', height=600)
fig.show()

### 💡 실무 예시: C와 gamma의 상호작용

- **C 크고 gamma 큼**: 과적합 위험 높음
- **C 작고 gamma 작음**: 과소적합 위험 높음
- **적절한 조합**: GridSearchCV로 탐색 필요

---

## 2.2 데이터 스케일링의 중요성

SVR은 **스케일링에 매우 민감**합니다. 스케일링 없이는 성능이 크게 저하됩니다.

In [ ]:
# 스케일링 유무에 따른 성능 비교
# 스케일링 없이
svr_no_scale = SVR(kernel='rbf', C=1.0, epsilon=0.1)
svr_no_scale.fit(X_train.values, y_train)
y_pred_no_scale = svr_no_scale.predict(X_test.values)

# 스케일링 적용
svr_scaled = SVR(kernel='rbf', C=1.0, epsilon=0.1)
svr_scaled.fit(X_train_scaled, y_train)
y_pred_scaled = svr_scaled.predict(X_test_scaled)

# 비교
print("=== 스케일링 효과 비교 ===")
print(f"스케일링 없음 - R2: {r2_score(y_test, y_pred_no_scale):.4f}, RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_no_scale)):.4f}")
print(f"스케일링 적용 - R2: {r2_score(y_test, y_pred_scaled):.4f}, RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_scaled)):.4f}")

### ⚠️ 주의: 왜 스케일링이 필수인가?

1. **거리 기반 계산**: RBF 커널은 데이터 간 거리를 사용
2. **특성 간 스케일 차이**: 스케일 큰 특성이 과도한 영향
3. **수치적 안정성**: 최적화 알고리즘의 수렴 속도 향상

---

## 2.3 GridSearchCV로 하이퍼파라미터 튜닝

In [ ]:
# 하이퍼파라미터 그리드 정의
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.01, 0.1, 1],
    'epsilon': [0.01, 0.1, 0.5]
}

# GridSearchCV
svr_base = SVR(kernel='rbf')
grid_search = GridSearchCV(
    svr_base,
    param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train_scaled, y_train)

print(f"\n최적 파라미터: {grid_search.best_params_}")
print(f"최적 CV RMSE: {np.sqrt(-grid_search.best_score_):.4f}")

In [ ]:
# 최적 모델로 테스트
best_svr = grid_search.best_estimator_
y_pred_best = best_svr.predict(X_test_scaled)

print("=== 튜닝된 SVR 성능 ===")
print(f"R2 Score: {r2_score(y_test, y_pred_best):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_best)):.4f}")

---

## 2.4 SVR vs 다른 회귀 모델 비교

In [ ]:
# 여러 모델 비교
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'SVR (기본)': SVR(kernel='rbf', C=1.0, epsilon=0.1),
    'SVR (튜닝)': best_svr,
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
}

results = []

for name, model in models.items():
    # 선형 모델과 SVR은 스케일링 데이터 사용
    if 'SVR' in name or name in ['Linear Regression', 'Ridge']:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        'Model': name,
        'RMSE': rmse,
        'R2': r2
    })

results_df = pd.DataFrame(results).sort_values('RMSE')
print("=== 모델 비교 결과 ===")
print(results_df.to_string(index=False))

In [ ]:
# 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=['RMSE (낮을수록 좋음)', 'R2 Score (높을수록 좋음)'])

fig.add_trace(
    go.Bar(x=results_df['Model'], y=results_df['RMSE'], marker_color='indianred'),
    row=1, col=1
)
fig.add_trace(
    go.Bar(x=results_df['Model'], y=results_df['R2'], marker_color='seagreen'),
    row=1, col=2
)

fig.update_layout(title='회귀 모델 성능 비교', height=400, showlegend=False)
fig.update_xaxes(tickangle=45)
fig.show()

### 💡 SVR 사용 가이드라인

| 상황 | SVR 추천도 | 대안 |
|------|-----------|------|
| 소규모 데이터 (< 10,000) | ⭐⭐⭐⭐⭐ | - |
| 중규모 데이터 (10,000~50,000) | ⭐⭐⭐ | Random Forest, XGBoost |
| 대규모 데이터 (> 50,000) | ⭐ | LightGBM, Neural Network |
| 비선형 패턴 | ⭐⭐⭐⭐⭐ | Random Forest |
| 이상치 많음 | ⭐⭐⭐⭐ | Huber Regression |

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 기본 SVR 학습 ⭐

**문제**: RBF 커널의 SVR 모델을 학습하고 R2 스코어를 계산하세요.

```python
from sklearn.svm import SVR
# X_train_scaled, X_test_scaled, y_train, y_test는 이미 준비됨
```

In [ ]:
# 여기에 코드를 작성하세요


### Q2. epsilon 설정 ⭐

**문제**: epsilon=0.5인 SVR 모델을 학습하고 RMSE를 계산하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q3. C 파라미터 비교 ⭐⭐

**문제**: C=0.1, 1, 10, 100 네 가지 값으로 SVR을 학습하고 R2 스코어를 비교하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q4. gamma 파라미터 실험 ⭐⭐

**문제**: gamma=0.01, 0.1, 1 세 가지 값으로 SVR을 학습하고 성능을 비교하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q5. 스케일링 효과 확인 ⭐⭐

**문제**: 스케일링 유무에 따른 SVR 성능 차이를 확인하세요. (스케일링 전/후 R2 출력)

In [ ]:
# 여기에 코드를 작성하세요


### Q6. 선형 커널 SVR ⭐⭐⭐

**문제**: kernel='linear'인 SVR을 학습하고 RBF 커널과 성능을 비교하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q7. 서포트 벡터 분석 ⭐⭐⭐

**문제**: SVR 모델의 서포트 벡터 수를 확인하고, C 값에 따라 어떻게 변하는지 분석하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q8. GridSearchCV로 최적화 ⭐⭐⭐⭐

**문제**: C=[0.1, 1, 10], gamma=[0.01, 0.1], epsilon=[0.1, 0.5]를 GridSearchCV로 최적화하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q9. 교차 검증 ⭐⭐⭐⭐

**문제**: cross_val_score로 SVR의 5-fold 교차 검증을 수행하고 평균 R2를 계산하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q10. 모델 비교 종합 ⭐⭐⭐⭐⭐

**문제**: SVR(튜닝), Linear Regression, Random Forest 세 모델을 비교하고 Plotly로 시각화하세요.

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: SVR 기초 요약

| 개념 | 설명 | 핵심 포인트 |
|------|------|------------|
| SVR | SVM의 회귀 버전 | ε-tube 내 오차 무시 |
| epsilon (ε) | 오차 허용 범위 | 클수록 더 많은 오차 허용 |
| C | 페널티 강도 | 클수록 오차에 엄격 |
| 스케일링 | 필수 전처리 | StandardScaler 권장 |

### Part 2: SVR 활용 요약

| 개념 | 설명 | 권장 값 |
|------|------|--------|
| RBF 커널 | 비선형 패턴 학습 | 기본 선택 |
| gamma | 영향 범위 | 'scale' 또는 GridSearch |
| GridSearchCV | 최적 파라미터 탐색 | C, gamma, epsilon 동시 탐색 |

### 💡 실무 팁

1. **스케일링 필수**: SVR 전에 반드시 StandardScaler 적용
2. **데이터 크기 확인**: 대용량(>50,000)에서는 SVR 비추천
3. **GridSearchCV 활용**: C, gamma, epsilon 세 파라미터 동시 최적화
4. **서포트 벡터 확인**: 너무 많으면 과적합, 너무 적으면 과소적합
5. **커널 선택**: 비선형이면 RBF, 선형이면 linear (더 빠름)